In [1]:
import requests
response = requests.get('https://www.rotowire.com/baseball/daily-lineups.php').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response, 'lxml')

In [3]:
import io
with io.open('today.html', 'w', encoding="utf-8") as f:
    f.write(str(soup.prettify()))
    
f.close()

In [4]:
print(soup.get_text())






MLB Daily Lineups | RotoWire










      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());

      gtag('config', 'UA-3226863-1', { 'link_attribution': true });
    


(function(w,d,t,r,u){var f,n,i;w[u]=w[u]||[],f=function(){var o={ti:"5295204"};o.q=w[u],w[u]=new UET(o),w[u].push("pageLoad")},n=d.createElement(t),n.src=r,n.async=1,n.onload=n.onreadystatechange=function(){var s=this.readyState;s&&s!=="loaded"&&s!=="complete"||(f(),n.onload=n.onreadystatechange=null)},i=d.getElementsByTagName(t)[0],i.parentNode.insertBefore(n,i)})(window,document,"script","//bat.bing.com/bat.js","uetq");






    window.addEventListener('load', function () {
        //load the apstag.js library
        !function(a9,a,p,s,t,A,g){if(a[a9])return;function q(c,r){a[a9]._Q.push([c,r])}a[a9]={init:function(){q("i",arguments)},fetchBids:function(){q("f",arguments)},setDisplayBids:function(){},targetingKeys:function(){return[]},_Q